<center><h1>写给不敢学习Transformer的人</h1> </center>
<center><h2>第一章: 祛魅：拆解机器思考的黑盒</h2> </center>



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JOSEFAzyq/fearless-transformer/blob/main/1.Demystifying-the-Black-Box.ipynb)

学习本章需要的背景知识:
- 基础初中数学
- 程序员 - 不需要!(当然有会更好)
- 不需要暂停阅读,去学习概念

一些简称:
- 模型: 特指 Transformer模型
- 3b1b: 3blue1brown 优秀的可视化学习类视频作家

学习本章你的收获:

- 顺利的理解Transformer的主要流程.
- 对机器学习产生兴趣.
- 自信的学完本章内容.

注意: 本章会省略很多细节,非完整Transformer流程介绍.

# 模型理解输入的过程

这不是机器学习的第一步,但是非常重要,所以在开始之前需要先理解这个.

## 多角度理解

在词典中,对一个词语的解释,往往是多角度的.它的词性,它在不同语境中的含义,它的音调等等等等

![yaoyaolingxian](./images/yaoyaolingxian.png)

甚至还可能包含一些背景故事,这使得一个词语能承载的信息远大于文字本身的意义.

我们写出一个词语很简单,但是想表达一个词语的含义就没那么容易了,总得想一想,用那么十几个字来解释.当遇到难以解释的情况时,就得用更多得词语来解释了.

这是我们遇到得第一个问题,也是Transformer需要解决的第一个难题:

**如何表达对于单词的理解**

也就是需要量化一个单词的语义,甚至量化一句话,一段文章的语义.

比如: 用户的输入是: 不要修改其他逻辑!只修改xxx文件.

模型对这句话的理解,除了一些硬性指令之外,它还会认为用户此时心情不好,是因为自己做错了事.

![](./images/transformer_view.png)

对于用户输入的理解,需要有一种量化的方式来记录.这就是计算机的魅力,它用数字做到了.对于多个维度的理解,恰好有个非常适合的数学工具来记录: 

**向量**

> 题外话: 中文的信息熵是英文的2倍以上,意味着同样的字符,中文能表达的内容往往远大于英文.

## 向量 Vector 
计算机向量来从多个维度表示一个单词,在模型中,使用 512 个维度来描述一个单词.
从人类的角度来看的话,这 512 个维度就像:

![](./images/queen.png)

这 512 个维度又叫做

**特征维度**

它描述的是最小单元的承载能力.

向量就是 1 维张量.没听过张量没关系.向量长这样: [ 3 , 6 , 9 , 13 ]

我们平时使用的数字,属于 0 维张量.

之后我们还会遇到 2 维张量,以及在机器学习中最常见的 3 维, 4 维张量.

![](./images/tensor_intro.png)

如果你是程序员的话,处理业务的时候,经常会处理 2 维数据,类似这样的数组对象:

`[{对象1},{对象2},{对象3}]`

## 把向量放到一起

向量是有非常明显的几何意义的.为了方便可视化,我们把特征维度缩放到 3 维.然后将一些单词根据向量的值放入到这个空间中:

![](./images/vector_space.png)

这里我们不过多去解释向量的几何意义,本章末尾会提供一些资料供你延展学习.

## 国王 + (女人 - 男人) = 女皇

来看一个经典的例子: King + (Women - Man) = Queen

![king_queen_man_women.gif](./images/king_queen_man_women.gif)

又或者另外一个经典的例子(来自3b1b的例子):

![](./images/what-is-hitler-plus-italy-minus-germany.png)

(答案是墨索里尼

## 前置动作: 将用户输入的语句转换为计算机的表达方式

在进入模型前的一步,是将输入语句转换为适合计算机的存储方式.

这一步很简单
1. 分词.将语句中的单词拆开.
2. 建立所有单词的唯一索引,就像一本词典.每个单词都有一个唯一ID.模型需要的只有这个ID,这个ID叫做 token_id.

![](./images/tokenizer.png)


也就是说,模型的输入,一个句子,是一个数组.如果是多个句子,那就是一个二维数组.

在机器学习中,为了提高效率,我们会尽可能的批量操作,也就是将句子拼成二维数组.所以即使是一个句子,也是按照批量操作的逻辑来.这个形状叫做 [批量长度,句子长度] ,一个10个单词的句子,就是 [1,10] 这样的形状.

不会单独处理一个句子: ["我","喜欢","深度学习"]  ,而是批量操作: [["我","喜欢","深度学习"]] 

图中的例子就是 **[3,4]** 这样的形状.句子长度按照最长的那一句话来,其他句子不够长度的想办法补齐.如何补齐我们之后会说.

![](./images/1_transformer_input.png)

这个步骤叫做 **Tokenizer** ,将输入 token 化.

**但是为了方便演示,本章之后的用词可能会倾向"单词",而不是token,这仅仅是为了更方便语义连贯.要记住,在模型中,不存在字符串,全是数字!**


> 得益于es等基础工具大力发展,分词工具已经很成熟了,几乎每个语种都发展出了自己的分词工具.分词不是简单的拆分字符,而是会根据词根去拆分.

# 第一步: 赋予每个单词(token)基础语义

现在我们正式进入到模型中来,并且准备好了一个输入矩阵.它的形状是 [3,4],记住这个形状,我们会在整个过程中不停的修改它.

![](./images/input_matrix.png)

我们要做的第一步,是单独赋予每个单词自己的语义.结合一下文章开始部分的内容,每个单词的含义是由 512 个维度 从 512 个角度来解释的.

这个过程叫做 **嵌入 Embedding** .

![](./images/embedding_1.png)

我们可以形象的认为,**嵌入**是将一个单词嵌入到了一个语义空间中,在这个语义空间中,它所嵌入到的位置,刚好是能完美表达它含义的位置.开头的内容中我们用3个维度来演示如何在空间中表达语义,在Transformer中,用512个维度来表达语义.

这 512 个维度叫做 **特征维度**,顾名思义,特征维度越大,能表达的特征越多.

**嵌入层**就是存放了整个词汇表语义的仓库(没有错,整个词汇表),单词通过**嵌入层**来索引到对应的语义.嵌入层也像是一个词典,通过 token_id 来返回对应的解释(也就是 512 个数字)

![](./images/embedding_layer.png)

OK,我们回到整个输入来说,模型输入的是一个 [3,4]形状的 token 矩阵.那么经过第一步 Embedding 后,每个 token 被 1 个 512 维度的向量所表示了.此时的矩阵理所当然的变化成了 [3,4,512] 这样的形状.

此时矩阵的形状变厚了.句子中的每个单词有了基础语义.

![](./images/embedding_output.png)

没有错,我们的数据输入现在变成了一个 3 维张量,它可以可视化成一个立体矩形(再多一个维度人类就无法可视化了),它的真实含义是[批量数量,句子长度,特征维度]

> 嵌入层是如何获取语义的,如何设定每个单词 512 个维度中的每个值的,如果你产生了这个疑问,很棒!这是机器学习的魅力,背后有非常有意思的机制.但是作为面向零基础的第一章我们不去详细解释它是如何实现的.而是在本章的最后,提供了一些供你课后学习的资料,google 提供了非常优质的学习内容.可以让你快速理解它的原理
> 现在,我们只需要知道嵌入层实现了这个功能,继续学习本章内容,不用急着去吃下太深的内容.

# 第二步: 理解上下文

也就是理解语句的本意.

## 注意力机制

现在的矩阵是一个 [3,4,512] 形状的 3 维矩阵. 第一维度是输入句子的数量,第二个维度是句子的单词数量,以最长的为准,第三个维度是特征维度.

现在来到最重要的步骤,在这个步骤中.模型会理解语句想表达的含义,也就是在独立单词的含义之上,加上了对上下文的理解.

于是需要接触到 Transformer 架构中最重要的一个环节: **注意力机制 (Attention)**

不要被这个看起来很抽象的词吓到! 所谓的注意力机制,其实就是将输入的语句矩阵,通过**公式**运算一下.来试图从中学习到一些关系以及更深层次的含义,比如 "机器学习很有趣","有趣"形容的是"机器学习".那么"机器学习"除了代表一种现代计算机的前沿算法,还加上了趣味性的含义.

![](./images/attention_flow.png)

这个公式就是 **注意力 (Attention) 公式**

模型通过注意力机制实现了理解上下文,可谓是惊天地泣鬼神的创举,是有多天才的人才能想到这个方法? 没错,就是google研究室的那些人,成天叨咕着捣鼓的发表论文,可他是真能出货啊.

注意力机制是将语句文本中的每一个单词互相发生关系,并且试图计算出单词之间的关系是否存在并且合理.

下图是对注意力机制的流程描述,如果你是零基础选手,看不懂是正常的,也不用看懂,在本章中,你需要知道的是,模型通过注意力机制,它有一套自己的流程公式,学会了如何理解语句.

**不要在张图片上过多停留,看10s感受一下就好!**

![](./images/attention_mechanism.jpg)

经过了这一步,我们的矩阵仍然保持着 [3,4,512] 的形状,但是此时的它,已经从保存每个 token 独立的含义,升级成了对语句的理解.

- 在第一步中,模型保存的是: 机器学习,是一种现代高科技计算机学科. AI,是人工智能.等等对每个单词的含义理解.
- 在第二步中,模型保存的是: 用户似乎正在学习机器学习,用户觉得机器学习很有趣,用户对机器学习敢兴趣,用户对AI也感兴趣.用户可能是程序员,用户喜欢互联网等等..

**没有错**,这是你可能接受不了的事实,就是此时模型对输入的理解,超越了单词本身的含义.

我们之前说过,计算机通过维度来表达对单词的理解.维度越多,能表达的信息越多(升维了,上帝视角总比当事人看的清楚).Transformer 通过自己的 512 个维度,可以对单词实现更多的理解,从而上升到对语句的理解,不仅仅停留在单词上.

![](./images/attention_dimention.jpg)

现在现在,我们的矩阵,拥有了上下文的理解.

> 作为一个零基础的入门章节,我一开始是想跳过注意力机制的,但它实在是太重要了,提前多了解一些总没错.


# 缓一缓
我们前进了非常多,在忽略了很多细节之后,获得了一个具有上下文语义的 token 矩阵.这意味着此时模型已经理解了用户的输入意图.
来回顾一下.模型做了哪些事:

1. 将单词拆分成 Token ,因为这是计算机表示单词的最佳方式.它既不占地方,又能高效率运算.这一步叫做 **Tokenizer**
2. 通过 **嵌入(Embedding)** 得到了一张基础矩阵.获取到了每个单词的基础语义.
3. 通过一些办法(**注意力机制(Attention)**),再基础矩阵之上,计算出了一张含有上下文含义的矩阵,这个矩阵已经可以完全理解这句话的意思了,在Transformer中这个矩阵叫做 **Memory**

是的,我们一直在做的事情,就是将一句没有任何意义的字符串,转换成计算机可以理解的数据.这个数据的构成一定是许多维度的描述才能实现的,本质上是在对人类的理解做**量化**处理.

![](./images/waitaminute.jpg)


## 背后的原理?

我们来聊点有意思的事情 - 模型如何进行学习.

我猜你很想知道模型是如何知道每个单词意味着什么,它如何推断出用户可能是个程序员,它背后的原理是什么,它藏了多少的数学秘密在里面.为什么让语句互相发生关系,就能学会含义了?

模型是通过训练来进行学习的,未经过训练的模型,就和刚出生的婴儿一样,对万物的认知大概率是错误的.

训练的结果本质上是**概率论**,语言模型的训练并不是获得一个准确的答案,而是获取一个概率 - 即 A 后面跟着的是 B 的概率.而这个概率,可以通过输入大量(非常大)的文本来进行分析和学习.一句话重复的次数多了,模型就会记住这个关系.

人类需要记忆来建立认知,模型通过数字来量化认知.模型建立认知的过程本质上和人类是一致的.

这些数字在模型中被称为 **参数(parameters)**,参数越多,模型表达能力越强.

本章内容不会展示深入的原理.然而,模型学习背后的原理-比想象中要简单粗暴很多.

简单来说，模型必须经过训练才能变聪明。训练的过程就三步：
- 猜：模型根据当前的参数，给出一个预测结果。
- 比：将预测结果和真实答案进行对比，算出差距（误差）。
- 改：根据这个差距，反向回去调整模型的参数。

![](./images/trainning_process.jpg)

对了,就是这么简单粗暴.没有什么神奇公式,全部都是机缘巧合.

至于具体怎么去对比,怎么去修正,怎么去知道需要修正什么.这些内容属于**机器学习**(Machine Learning)的范围,而 Transformer 是建立在机器学习之上的架构.

在机器学习中,修正的过程叫做**反向传播(back)**,如何去修正依靠的是**梯度下降(gradient desent)**.Transformer使用的学习方式叫做**监督式学习(supervice)**,涉及到的**回归()**模型大部分是**线性模型(linear)**以及**神经网络(nerual network)**.

你可能或多或少听过这些词并且不知道什么意思,没关系,我们之后的章节会让你很轻松的理解这些看起来不明所以的单词.继续看下去就行.

# 第三步: 输出内容

## 输出

之前,模型理解了用户的输入,在模型中生成了 Memory 这样一个带有上下文语义以及理解的矩阵.它是一个 (句子数量,token数量,理解维度) 这样一个形状的矩阵,也就是[3,4,512].

现在轮到模型准备输出内容了

又要告诉你一件很有趣的事情了! 模型并不是直接吐出单词,而是先构思语义,再转换为文字的!

模型从零开始构思语义,一点点的加进去.最后形成完整的语义,再将语义转换为文字;

语义的承载体同样是矩阵,我们之前生成的 Memory 就是对输入的理解.现在我们需要生成一个对输出的理解. (这也体现了,模型在运行过程中,内部运算的对象其实是语义,而并非确定的单词)

就像人类说话一样,先产生意图,比如,我要让她喜欢我;我要表现的专业一点;好像有危险,我得小心一点说话.

![image.png](./images/output_flow.jpg)

在理解输入的过程中,模型只做了一件事,就是让输入语句互相发生关系,来学习理解输入的含义.

而在输出的时候,除了输出的语句自行理解,还需要使用到 Memory ,来保证输出和输入是有语义关系的.也就是为了避免牛头不对马嘴的情况.

之前提到过注意力机制.其实是分为两种.
- 一种就是在理解上下文中,内容自行理解,上下文就是它本身.这种叫做**自注意力(Self-Attention)**.模型在理解输入,和进行输出时,都有用到这个机制.
- 一种是在输出过程中,输出内容结合 Memory,一起来进行理解,这种交错**交叉注意力(Cross-Attention)**,上下文除了输出本身,还包括输入.模型只有在输出时,才会用到这个机制.

![image.png](./images/self_cross_attention.jpg)

输出过程中,模型每次输出一个意图 - 没有错,此时它还不是具体的 token ,只是一个意图,但是它的格式和 token 一样,都是背后藏着 512 个特征参数的单元.

输出产生的意图,只要不是结束.就会循环产生(没有错,就是for循环),直到产生的意图是结束.

输出所产生的意图,会再次成为输出流程中的输入,不停的追加.这个过程叫做**自回归(Auto-Regressive)**

![image.png](./images/autoregressive.jpg)

每经过一次输出过程,模型都会输出一个意图,意图的形状是 [1,512].没有错,就是所有特征维度.然后作为输入,再次进入到输出过程.生成新的[1,512],拼接到原始输入上,变成[2,512],如此往复,直到结束意图的产生.



# 最后一步,将意图转为单词

第三步中,模型输出的是一个个的意图.

在第一步中,模型输入的是语句,模型将语句转换为token再转换为意图.这个过程叫做嵌入 Embedding.

输出时,模型输出的是意图,现在需要再将意图转换为token,这个过程叫做 **UnEmbedding** ,可以翻译成反嵌入.

![image.png](./images/unembedding.jpg)

将所有意图转换为单词,模型的任务就完成了.我们平时看到模型一个一个字吐出来,就是模型再将意图一个一个转换为单词.

恭喜你!到这一步,就是 Transformer 架构的全流程.我们忽略了一些细节,但是 Transformer 的架构就是如此清晰!

在本章中,我们知道了**嵌入**这一个非常重要的概念,它量化了语义.了解到了模型的基本工作流程:
1. 量化语义
2. 理解输入
3. 结合输入进行输出

接触到了一些概念,这些名词虽然抽象但是又很合理.
- 注意力机制: 让语句互相了解,建立关系.
- 自回归: 将输出作为输入,循环往复.
- 嵌入: 将意图嵌入到语义空间中.
- 特征维度: 描述意图的角度.

在之后的学习中,你会发现各种模型架构均由多层(Layer)组成的,每一层有自己的职责所在.我们甚至可以调整每一层的实现方式,来达到不同效果.

在 Transformer 中,理解上下文的模块,被成为 Encoder.输出模块,被成为 Decoder.Encoder 和 Decoder 的内部由各种层组装而来.

![image.png](./images/simpletransformer.jpg)


# 额外内容: 训练和推理

模型,指的是一个架构.内部通过不同层,不同算法组成.层中含有参数,参数经过训练才有意义.

一个标准的 Transformer 架构中的参数有 9000多万个.训练的目的,就是将这 9000多万的参数调整成我们需要的样子.

训练和推理是 2 个不一样的场景.流程也有不同的地方.

我们日常使用接触的是推理.模型一个一个地输出单词.

我们看不到的是训练过程. Transformer 训练的方法,是输入一句不完整的句子,让模型推理这句话跟着的单词是什么.再将推理结果和完整的句子进行比较.再通过比较结果来反向调整参数的过程.

前面内容中的举得输入的例子,它是由 3 句话组成的 [3,4,512]形状的张量.是因为模型可以并行的去进行推理,同时推理三句话.得益于现代硬件的升级,并行训练的能力提升非常大,使得训练速度快了非常多.

![image.png](./images/trainandinference.jpg)

# 课后学习(建议按顺序来)

1. [建议]一定要去看看 3blue1brown的视频.[直观解释Transformer](https://www.bilibili.com/video/BV13z421U7cs/?spm_id_from=333.1387.upload.video_card.click&vd_source=6f64228d557b66be8e52e4682c185a12).
里面不仅介绍了Transformer,还从语义和几何的角度解释了向量

2. [建议]学习google的机器学习中的线性回归课程,非常简单,能让你建立起对机器学习的基础认识.[线性回归](https://developers.google.com/machine-learning/crash-course/linear-regression?hl=zh-cn).如果有额外精力,把其他的课程看完也行.

3. [建议]去看 3blue1brown 线性代数的入门视频,非常推荐. 会从几何角度让你认识矩阵.

4. [必须]了解矩阵是什么,对于矩阵的操作.基本运算,维度变化.学习 pytorch关于矩阵的操作.不需要很深的线性代数的知识,但是对矩阵的操作要非常熟练.最全的资料:[PyTorch官网.Tensor](https://docs.pytorch.org/docs/stable/tensors.html)


提示: 去接触google 的 colab ,能让你快速完成实验.不推荐早期学习阶段在本地搭建环境.如果一定要本地搭建环境的话推荐用Miniforge进行环境管理和依赖安装.